In [ ]:
from collections import deque
from datasets import load_dataset
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import kendalltau
import spacy
import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# Setup
Loading in model (we use GPT-2)

In [ ]:
model_name = "gpt2"
config = AutoConfig.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, config=config)
nlp = spacy.load("en_core_web_sm")

# Set pad_token to eos_token for GPT-2 tokenizer to avoid padding errors
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def get_sentence_token_spans(text, tokenizer):
    """
    Splits text into sentences and maps them to token ranges.
    Returns: List of (sentence_text, start_token_idx, end_token_idx)
    """
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    # Tokenize the full text with offsets to align
    inputs = tokenizer(text, return_tensors="pt", return_offsets_mapping=True)
    offset_mapping = inputs.offset_mapping[0].numpy() # (num_tokens, 2)

    sent_spans = []
    current_sent_idx = 0

    # Simple alignment: Find which tokens fall into the char range of the sentence
    # Note: This is a greedy alignment; production code needs robust char-to-token mapping
    char_cursor = 0
    for sent in sentences:
        start_char = text.find(sent, char_cursor)
        end_char = start_char + len(sent)
        char_cursor = end_char

        # Find tokens that overlap with this sentence range
        sent_token_indices = []
        for i, (start, end) in enumerate(offset_mapping):
            # Check overlap
            if start >= start_char and end <= end_char:
                sent_token_indices.append(i)

        if sent_token_indices:
            sent_spans.append({
                "text": sent,
                "start": sent_token_indices[0],
                "end": sent_token_indices[-1]
            })

    return inputs, sent_spans

Tester for the above code

In [ ]:
text = (
    "The CPU handles the instructions. "
    "The RAM stores the data temporarily. "
    "However, the GPU is needed for rendering images. "
    "Together, these components power the computer. "
    "The user interacts via the keyboard."
)

inputs, sent_data = get_sentence_token_spans(text, tokenizer)
print(f"Detected {len(sent_data)} sentences.")

The below code runs the model once, extracts the attention, and then changes the attention from a (tokens x tokens) matrix to the sentence matrix (sentences x sentences).

In [ ]:
def extract_sentence_dependency_matrix(model, inputs, sent_data, agg_method='mean'):
    """
    Runs the model and aggregates token-level attention into sentence-level attention.
    """
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)

    # outputs.attentions is a tuple of (num_layers) tensors of shape (batch, heads, seq, seq)
    # We will average across all layers and heads for a 'global' attention view.
    # You could also target specific 'induction heads' if you wanted to be fancy.

    # Stack layers -> (layers, batch, heads, seq, seq)
    all_attentions = torch.stack(outputs.attentions)

    # Average over layers and heads -> (seq, seq)
    # Squeeze batch dimension (0)
    avg_attention = all_attentions.mean(dim=(0, 2)).squeeze(0).cpu().numpy()

    n_sentences = len(sent_data)
    sent_attn_matrix = np.zeros((n_sentences, n_sentences))

    for i in range(n_sentences): # Query Sentence (The one 'looking back')
        for j in range(n_sentences): # Key Sentence (The one being looked at)

            # Get token ranges
            q_start, q_end = sent_data[i]['start'], sent_data[i]['end']
            k_start, k_end = sent_data[j]['start'], sent_data[j]['end']

            # Extract sub-matrix for these two sentences
            # Note: causal masking means attn is 0 if k > q, handled naturally by the model
            sub_matrix = avg_attention[q_start:q_end+1, k_start:k_end+1]

            if sub_matrix.size > 0:
                if agg_method == 'max':
                    score = np.max(sub_matrix)
                elif agg_method == 'mean':
                    score = np.mean(sub_matrix)
                elif agg_method == 'sum':
                    score = np.sum(sub_matrix)

                sent_attn_matrix[i, j] = score

    return sent_attn_matrix

In [ ]:
attn_matrix = extract_sentence_dependency_matrix(model, inputs, sent_data, agg_method='mean')

In [ ]:
# Visualization of Sentence Attention
plt.figure(figsize=(8, 6))
plt.imshow(attn_matrix, cmap='viridis')
plt.title("Sentence-to-Sentence Attention (Raw)")
plt.xlabel("Key Sentence (Source Info)")
plt.ylabel("Query Sentence (Destination Info)")
plt.colorbar()
plt.show()

Now, we construct the DAG of the attentions. In this DAG, there's an edge between two sentences if the attention is above some certain threshold. This is because any two sentences will likely have some small attention, so we only want to filter for sentence pairs that are highly important to each other.

In [ ]:
def generate_permutations(sent_data, attn_matrix, quantile_threshold=0.2, n_samples=5):
    """
    Builds the dependency DAG and samples 'n_samples' unique permutations.
    """
    G = nx.DiGraph()
    n = len(sent_data)

    # 1. Add Nodes
    for i in range(n):
        G.add_node(i, text=sent_data[i]['text'])

    # 2. Determine Threshold
    lower_triangle = attn_matrix[np.tril_indices(n, k=-1)]
    if len(lower_triangle) > 0:
        cutoff = np.quantile(lower_triangle, quantile_threshold)
    else:
        cutoff = 0.0

    # 3. Add Edges (Pruning weak attention)
    # Edge j -> i means i depends on j
    for i in range(n): # Later sentence
        for j in range(i): # Earlier sentence
            if attn_matrix[i, j] > cutoff:
                G.add_edge(j, i)

    # 4. Sample Permutations
    unique_permutations = set()
    attempts = 0
    max_attempts = n_samples * 5 # Stop if we can't find enough unique ones

    # Add the original sequence as the first baseline "permutation"
    original_seq = tuple(range(n))
    unique_permutations.add(original_seq)

    while len(unique_permutations) < (n_samples + 1) and attempts < max_attempts:
        # Call the internal sampler
        new_perm = sample_topological_sort(G)
        unique_permutations.add(tuple(new_perm))
        attempts += 1

    # Convert sets of tuples back to lists
    results = [list(p) for p in unique_permutations]

    return G, results

Since there's $O(n!)$ possible permutations, we limit the permutations we use to some random sample of all valid topological ordering of the vertices of the DAG.

In [ ]:
def sample_topological_sort(G):
    """
    Generates a single random topological sort of a DAG using
    Randomized Kahn's Algorithm.

    Args:
        G: A NetworkX DiGraph.
    Returns:
        A list of node indices representing a valid permutation.
    """
    in_degree = {u: d for u, d in G.in_degree() if d > 0}

    # Initialize queue with nodes having 0 in-degree (no dependencies)
    zero_in_degree_queue = [n for n in G.nodes() if G.in_degree(n) == 0]

    sorted_order = []

    while zero_in_degree_queue:
        # Pick a random candidate from the available "next steps"
        candidate_idx = random.randrange(len(zero_in_degree_queue))
        current_node = zero_in_degree_queue.pop(candidate_idx)

        sorted_order.append(current_node)

        # 'Virtual' removal of the node: update neighbor in-degrees
        for neighbor in G.successors(current_node):
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                zero_in_degree_queue.append(neighbor)
                del in_degree[neighbor]

    # Cycle detection fallback
    if len(sorted_order) != len(G.nodes()):
        # This usually means the graph had a cycle.
        # Fallback: return the nodes sorted by their original index (0, 1, 2...)
        return sorted(list(G.nodes()))

    return sorted_order

In [ ]:
dag, permutations = generate_permutations(
    sent_data,
    attn_matrix,
    quantile_threshold=0.5,
    n_samples=3
)

print(f"Generated {len(permutations)} unique sequences (including original).\n")

Printing out 3 examples of the generated sentences:

In [ ]:
for i, perm in enumerate(permutations[:3]):
    print(f"--- Permutation {i+1} ---")
    reconstructed_text = " ".join([sent_data[idx]['text'] for idx in perm])
    print(reconstructed_text)
    print("")

Here's a visualization of what the DAG looks like for this example.

In [ ]:
plt.figure(figsize=(8, 4))
pos = nx.spring_layout(dag)
nx.draw(dag, pos, with_labels=True, node_color='lightblue', edge_color='gray', arrowsize=20)
plt.title("Semantic Dependency DAG")
plt.show()

## Real Data Example
Now that we have a working setup, we can try this on some synthetic data!

In [ ]:
def get_data_loader(dataset_name="wikitext", subset="wikitext-103-raw-v1", split="train"):
    """
    Returns an iterable streaming dataset.
    """
    # streaming=True means we don't download the 100GB+ file to disk
    dataset = load_dataset(dataset_name, subset, split=split, streaming=True)
    return dataset

In [ ]:
# Initialize loader
raw_dataset = get_data_loader()

# Create a generator to fetch documents one by one
data_iter = iter(raw_dataset)

Note: you need to run the below cell multiple times until you get something that actually looks like part of a Wikipedia article.

In [ ]:
print("Fetching document from Wikitext-103...")
sample = next(data_iter)
doc_text = sample['text']
print(f"\nOriginal Doc Start: {doc_text[:100]}...\n")

Generating the permutations!

In [ ]:
inputs, sent_data = get_sentence_token_spans(doc_text, tokenizer)
print(f"Detected {len(sent_data)} sentences.")

attn_matrix = extract_sentence_dependency_matrix(model, inputs, sent_data)
dag, permutations = generate_permutations(sent_data, attn_matrix, quantile_threshold=0.67)

print(f"Generated {len(permutations)} permutations (including original).")

In [ ]:
for i, perm in enumerate(permutations[:3]):
    print(f"--- Permutation {i+1} ---")
    reconstructed_text = " ".join([sent_data[idx]['text'] for idx in perm])
    print(reconstructed_text)
    print("")

In [ ]:
plt.figure(figsize=(8, 4))
pos = nx.spring_layout(dag)
nx.draw(dag, pos, with_labels=True, node_color='lightblue', edge_color='gray', arrowsize=20)
plt.title("Semantic Dependency DAG")
plt.show()

# Training

## Setup for training

Model training hyperparameters:

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 5e-5
NUM_STEPS = 1000
PERMUTATIONS_PER_DOC = 3 # K samples
ATTENTION_THRESHOLD = 0.67

Getting ready to train

In [ ]:
model.to(DEVICE)
model.train()

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

running_loss = 0.0
total_tokens_processed = 0
start_time = time.time()

print(f"Starting training on {DEVICE}...")
print(f"Strategy: 1 Original + {PERMUTATIONS_PER_DOC} Synthetic per step.")

dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
data_iter = iter(dataset)

## Original Training loop

In [ ]:
for step in range(NUM_STEPS):
    try:
        # 1. Fetch Data
        sample = next(data_iter)
        text = sample['text']

        # Skip garbage/short data
        if len(text) < 200:
            continue

        # 2. Preprocessing & Alignment
        # We need the inputs on the device for the 'diagnostic' pass
        inputs, sent_data = get_sentence_token_spans(text, tokenizer)
        if len(sent_data) < 5: # Need enough sentences to permute
            continue

        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # 3. Diagnostic Pass (No Grad)
        # We put model in eval temporarily to get clean attention maps
        model.eval()
        attn_matrix = extract_sentence_dependency_matrix(model, inputs, sent_data)
        model.train() # Switch back to train

        # 4. Generate Synthetic Data
        # Returns [Original_Indices, Perm1, Perm2...]
        _, permutations = generate_permutations(
            sent_data,
            attn_matrix,
            quantile_threshold=ATTENTION_THRESHOLD,
            n_samples=PERMUTATIONS_PER_DOC
        )

        # 5. Prepare Training Batch
        # We re-tokenize the permutations.
        # Why? Because BPE merges might change when sentence neighbors change.
        batch_texts = []
        for perm_indices in permutations:
            # Reconstruct string
            perm_text = " ".join([sent_data[idx]['text'] for idx in perm_indices])
            batch_texts.append(perm_text)

        # Tokenize the batch of (1 Real + K Synthetic)
        batch_inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(DEVICE)

        # 6. Training Step
        optimizer.zero_grad()

        # Forward pass on the mixed batch
        outputs = model(**batch_inputs, labels=batch_inputs["input_ids"])
        loss = outputs.loss

        loss.backward()

        # Gradient clipping to prevent exploding gradients from bad permutations
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # 7. Logging
        current_loss = loss.item()
        running_loss += current_loss
        batch_tokens = batch_inputs["input_ids"].numel()
        total_tokens_processed += batch_tokens

        if step % 10 == 0:
            if len(permutations) > 1:
              original_indices = permutations[0]
              synthetic_indices = permutations[1]

              # Calculate Kendall Tau
              # 1.0 = Same order, -1.0 = Reverse order, 0.0 = Random
              tau, _ = kendalltau(original_indices, synthetic_indices)

              # Convert to a "Scramble Score" (0 = Identical, 1 = Very scrambled)
              # Using (1 - tau) / 2 to normalize
              scramble_score = (1 - tau) / 2

              print(f"   > Debug: Scramble Score: {scramble_score:.2f} (Tau: {tau:.2f})")

            avg_loss = running_loss / 10 if step > 0 else current_loss
            elapsed = time.time() - start_time
            tokens_per_sec = total_tokens_processed / elapsed

            print(f"Step {step} | Loss: {avg_loss:.4f} | Tok/s: {tokens_per_sec:.0f} | "
                  f"Permutations: {len(batch_texts)}")
            running_loss = 0.0

    except StopIteration:
        print("Dataset exhausted.")
        break
    except Exception as e:
        print(f"Skipping step due to error: {e}")
        continue

print("Training complete.")

## Dynamic Threshold

 In the beginning, the model's attention maps are messy and not very semantic.

* **Concept:** Start with a `quantile_threshold` of **0.9** (very strict, graph is almost linear, very few permutations).

* **Evolution:** Linearly decrease the threshold to **0.2** over the course of training.

* **Why:** As the model learns better causal dependencies, its attention matrix becomes cleaner. You can then trust it to allow more aggressive permutations later in training.

In [ ]:
def get_dynamic_threshold(current_step, total_steps, start_val=0.9, end_val=0.2):
    """
    Linearly anneals the threshold from start_val down to end_val.
    """
    if current_step >= total_steps:
        return end_val

    progress = current_step / total_steps
    current_val = start_val - (progress * (start_val - end_val))
    return current_val

## Discriminator Rejection Sampling

Blindly trusting the topological sort is risky. Some sorts might preserve *dependencies* but break *discourse coherence* (e.g., "However," appearing at the start).

* **Add a small Perplexity Filter:**

    1. Generate 10 topological sorts.

    2. Run a tiny, fixed model (e.g., `distilgpt2`) or the current model itself (with `no_grad`) on them.

    3. Calculate the perplexity of the generated sentences.

    4. **Discard** any permutation where `Perplexity > 2.0 * Original_Perplexity`.

    5. Train only on the "plausible" synthetic data.


In [ ]:
judge_name = "distilgpt2"
judge_tokenizer = AutoTokenizer.from_pretrained(judge_name)
judge_model = AutoModelForCausalLM.from_pretrained(judge_name).to(DEVICE)
judge_model.eval()

def calculate_perplexity(text, model, tokenizer):
    """Calculates perplexity (PPL) of a string."""
    encodings = tokenizer(text, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model(**encodings, labels=encodings["input_ids"])
    return torch.exp(outputs.loss).item()

## Contrastive Consistency Loss

Instead of just treating the synthetic data as "more text to predict," treat it as a **semantic anchor**.

* **Idea:** The embedding of the *last token* of the document should be similar for the Original and the Permutation (since they theoretically mean the same thing).

* **Implementation:** Add a loss term: $L_{total} = L_{CLM} + \lambda \cdot MSE(H_{original}, H_{permuted})$.

* **Why:** This forces the model to learn representations that are robust to sentence reordering, effectively teaching it to ignore "positional noise."

## Soft-Edge Sampling (Probabilistic DAG)

Currently, we binary threshold the edges (Edge exists OR it doesn't).

* **Idea:** Treat the attention score as a probability $P(edge)$.

* **Algorithm:** When building the DAG, add edge $A \to B$ with probability proportional to `Attention(B, A)`.

* **Effect:** This naturally handles "maybe" dependencies. Sometimes the graph will be strict, sometimes loose. This adds noise to the structure generation itself, which acts as a stronger regularizer.


In [ ]:
def generate_permutations_probabilistic(sent_data, attn_matrix, n_samples=3):
    """
    Builds a DAG where edges are added probabilistically based on attention strength.
    """
    G = nx.DiGraph()
    n = len(sent_data)
    for i in range(n):
        G.add_node(i, text=sent_data[i]['text'])

    # 1. Get stats to normalize probabilities
    lower_triangle = attn_matrix[np.tril_indices(n, k=-1)]

    if len(lower_triangle) > 0:
        min_attn = np.min(lower_triangle)
        max_attn = np.max(lower_triangle)
        range_attn = max_attn - min_attn + 1e-9 # Avoid div/0
    else:
        # Fallback for single sentence or empty matrix
        return G, [list(range(n))]

    # 2. Add Edges Probabilistically
    for i in range(n):       # Later sentence
        for j in range(i):   # Earlier sentence
            raw_score = attn_matrix[i, j]

            # Normalize score to 0.0 - 1.0 range (Min-Max scaling)
            # Stronger attention = Higher probability of edge existence
            prob = (raw_score - min_attn) / range_attn

            # Sampling: Roll the dice
            if random.random() < prob:
                G.add_edge(j, i)

    # 3. Sample from this specific graph instance
    # Note: With soft edges, you might get a different graph every time.
    # Ideally, you regenerate the Graph K times, or sample K sorts from this one Graph.
    # Here we sample K sorts from this one instance.

    unique_permutations = set()
    unique_permutations.add(tuple(range(n))) # Always keep original

    attempts = 0
    while len(unique_permutations) < (n_samples + 1) and attempts < (n_samples * 10):
        new_perm = sample_topological_sort(G) # Use the sampler function from previous step
        unique_permutations.add(tuple(new_perm))
        attempts += 1

    return G, [list(p) for p in unique_permutations]

## New Training Loop

In [ ]:
for step in range(NUM_STEPS):
    try:
        # 1. Fetch Data
        sample = next(data_iter)
        text = sample['text']

        # Skip garbage/short data
        if len(text) < 200:
            continue

        # 2. Preprocessing & Alignment
        # We need the inputs on the device for the 'diagnostic' pass
        inputs, sent_data = get_sentence_token_spans(text, tokenizer)
        if len(sent_data) < 5: # Need enough sentences to permute
            continue

        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # 3. Diagnostic Pass (No Grad)
        # We put model in eval temporarily to get clean attention maps
        model.eval()
        attn_matrix = extract_sentence_dependency_matrix(model, inputs, sent_data)
        model.train() # Switch back to train

        # generate synthetic data
        current_threshold = get_dynamic_threshold(step, NUM_STEPS)
        _, permutations = generate_permutations(
            sent_data,
            attn_matrix,
            quantile_threshold=current_threshold,
            n_samples=PERMUTATIONS_PER_DOC
        )

        # 5. Prepare Training Batch
        # We re-tokenize the permutations.
        # Why? Because BPE merges might change when sentence neighbors change.
        batch_texts = []
        for perm_indices in permutations:
            # Reconstruct string
            perm_text = " ".join([sent_data[idx]['text'] for idx in perm_indices])
            batch_texts.append(perm_text)

        # Tokenize the batch of (1 Real + K Synthetic)
        batch_inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(DEVICE)

        # 6. Training Step
        optimizer.zero_grad()

        # Forward pass on the mixed batch
        outputs = model(**batch_inputs, labels=batch_inputs["input_ids"])
        loss = outputs.loss

        loss.backward()

        # Gradient clipping to prevent exploding gradients from bad permutations
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # 7. Logging
        current_loss = loss.item()
        running_loss += current_loss
        batch_tokens = batch_inputs["input_ids"].numel()
        total_tokens_processed += batch_tokens

        if step % 10 == 0:
            if len(permutations) > 1:
              original_indices = permutations[0]
              synthetic_indices = permutations[1]

              # Calculate Kendall Tau
              # 1.0 = Same order, -1.0 = Reverse order, 0.0 = Random
              tau, _ = kendalltau(original_indices, synthetic_indices)

              # Convert to a "Scramble Score" (0 = Identical, 1 = Very scrambled)
              # Using (1 - tau) / 2 to normalize
              scramble_score = (1 - tau) / 2

              print(f"   > Debug: Scramble Score: {scramble_score:.2f} (Tau: {tau:.2f})")

            avg_loss = running_loss / 10 if step > 0 else current_loss
            elapsed = time.time() - start_time
            tokens_per_sec = total_tokens_processed / elapsed

            print(f"Step {step} | Loss: {avg_loss:.4f} | Tok/s: {tokens_per_sec:.0f} | "
                  f"Permutations: {len(batch_texts)}")
            running_loss = 0.0

    except StopIteration:
        print("Dataset exhausted.")
        break
    except Exception as e:
        print(f"Skipping step due to error: {e}")
        continue

print("Training complete.")